<a href="https://colab.research.google.com/github/MtHenriqueF/Customer-segmentation-clustering/blob/Analise-exploratoria/clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução

Usaremos os arquivos CSV disponíveis em: [customer-segmentation](https://www.kaggle.com/datasets/kaushiksuresh147/customer-segmentation-classification)

Esse dataset foi criado para classificação entre grupos A, B, C e D, porém utilizarei ele para aplicar modelo de clustering, segue uma breve descrição:


#### **Context**
An automobile company has plans to enter new markets with their existing products (P1, P2, P3, P4, and P5). After intensive market research, they’ve deduced that the behavior of the new market is similar to their existing market.

In their existing market, the sales team has classified all customers into 4 segments (A, B, C, D ). Then, they performed segmented outreach and communication for a different segment of customers. This strategy has work e exceptionally well for them. They plan to use the same strategy for the new markets and have identified 2627 new potential customers.

You are required to help the manager to predict the right group of the new customers.



# Carregando e configurando dataset

## Carregando

In [3]:
import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("kaushiksuresh147/customer-segmentation")

print("Path to dataset files:", path)

print("Arquivos disponíveis:")
print(os.listdir(path))


Path to dataset files: /kaggle/input/customer-segmentation
Arquivos disponíveis:
['Train.csv', 'Test.csv']


## Combinando treino e teste

In [4]:
import pandas as pd
import os


train_path = os.path.join(path, "Train.csv")
test_path = os.path.join(path, "Test.csv")

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)



In [5]:
print("--- Informações Iniciais ---")
print(f"Shape do df_train: {df_train.shape}")
print(f"Shape do df_test: {df_test.shape}")
print("\nColunas em Train.csv:")
print(df_train.columns)
print("\nColunas em Test.csv:")
print(df_test.columns)


--- Informações Iniciais ---
Shape do df_train: (8068, 11)
Shape do df_test: (2627, 11)

Colunas em Train.csv:
Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

Colunas em Test.csv:
Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')


In [7]:
df_combined = pd.concat([df_train, df_test], ignore_index=True)

print(f"Shape do df_combined: {df_combined.shape}")
print(f"Total de linhas esperado: {df_train.shape[0] + df_test.shape[0]}")



--- Informações Após a Junção ---
Shape do df_combined: (10695, 11)
Total de linhas esperado: 10695


## Removendo ID

ID é um péssimo atributo para qualquer modelo não supervisionado, pois não características de agrupamento que podem ser tiradas de um atributo que deve ser único para qualquer linha

In [17]:
df = df_combined.drop("ID", axis=1)

# Analise exploratória

In [19]:
print("\nInformações gerais do DataFrame combinado:")
df.info()


Informações gerais do DataFrame combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           10695 non-null  object 
 1   Ever_Married     10505 non-null  object 
 2   Age              10695 non-null  int64  
 3   Graduated        10593 non-null  object 
 4   Profession       10533 non-null  object 
 5   Work_Experience  9597 non-null   float64
 6   Spending_Score   10695 non-null  object 
 7   Family_Size      10247 non-null  float64
 8   Var_1            10587 non-null  object 
 9   Segmentation     10695 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 835.7+ KB


In [10]:
df_combined.describe()


,ID,Age,Work_Experience,Family_Size
count,10695.000000,10695.000000,9597.000000,10247.000000
mean,463468.088640,43.511828,2.619777,2.844052
std,2600.966411,16.774158,3.390790,1.536427
min,458982.000000,18.000000,0.000000,1.000000
25%,461220.500000,30.000000,0.000000,2.000000
50%,463451.000000,41.000000,1.000000,3.000000
75%,465733.500000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [11]:
df_combined.describe(include='object')


,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1,Segmentation
count,10695,10505,10593,10533,10695,10587,10695
unique,2,2,2,9,3,7,4
top,Male,Yes,Yes,Artist,Low,Cat_6,D
freq,5841,6163,6570,3318,6494,6910,3027


In [12]:
df_combined.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [13]:
print(df_combined.isnull().sum())


,0
ID,0
Gender,0
Ever_Married,190
Age,0
Graduated,102
Profession,162
Work_Experience,1098
Spending_Score,0
Family_Size,448
Var_1,108


In [14]:
df_combined.nunique()

,0
ID,8363
Gender,2
Ever_Married,2
Age,67
Graduated,2
Profession,9
Work_Experience,15
Spending_Score,3
Family_Size,9
Var_1,7
